In [78]:
import requests
from funcs import utils, paths
from funcs.data_processing import stage1_processing

```
curl -X POST --data-binary @[Inputfile] https://www.ncbi.nlm.nih.gov/research/pubtator-api/annotations/annotate/submit/[Bioconcept]
```

```
curl https://www.ncbi.nlm.nih.gov/research/pubtator-api/annotations/annotate/retrieve/[SessionNumber]
```

## simple test

In [79]:
annotate_url_template = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/annotations/annotate/submit/{concept_type}"
session_url_template = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/annotations/annotate/retrieve/{session_id}"

In [80]:
BIOCONCEPT = "All"

In [81]:
TEXT = """\
Autosomal-dominant striatal degeneration (ADSD) is an autosomal-dominant movement disorder affecting the striatal part of the basal ganglia. ADSD is characterized by bradykinesia, dysarthria, and muscle rigidity. These symptoms resemble idiopathic Parkinson disease, but tremor is not present. 
-*-
Using genetic linkage analysis, we have mapped the causative genetic defect to a 3.25 megabase candidate region on chromosome 5q13.3-q14.1. A maximum LOD score of 4.1 (Theta = 0) was obtained at marker D5S1962. Here we show that ADSD is caused by a complex frameshift mutation (c.94G>C+c.95delT) in the phosphodiesterase 8B (PDE8B) gene, which results in a loss of enzymatic phosphodiesterase activity. We found that PDE8B is highly expressed in the brain, especially in the putamen, which is affected by ADSD. PDE8B degrades cyclic AMP, a second messenger implied in dopamine signaling. Dopamine is one of the main neurotransmitters involved in movement control and is deficient in Parkinson disease. We believe that the functional analysis of PDE8B will help to further elucidate the pathomechanism of ADSD as well as contribute to a better understanding of movement disorders.
"""
TEXT

'Autosomal-dominant striatal degeneration (ADSD) is an autosomal-dominant movement disorder affecting the striatal part of the basal ganglia. ADSD is characterized by bradykinesia, dysarthria, and muscle rigidity. These symptoms resemble idiopathic Parkinson disease, but tremor is not present. \n-*-\nUsing genetic linkage analysis, we have mapped the causative genetic defect to a 3.25 megabase candidate region on chromosome 5q13.3-q14.1. A maximum LOD score of 4.1 (Theta = 0) was obtained at marker D5S1962. Here we show that ADSD is caused by a complex frameshift mutation (c.94G>C+c.95delT) in the phosphodiesterase 8B (PDE8B) gene, which results in a loss of enzymatic phosphodiesterase activity. We found that PDE8B is highly expressed in the brain, especially in the putamen, which is affected by ADSD. PDE8B degrades cyclic AMP, a second messenger implied in dopamine signaling. Dopamine is one of the main neurotransmitters involved in movement control and is deficient in Parkinson disea

In [82]:
url = annotate_url_template.format(concept_type=BIOCONCEPT)
r = requests.post(url, data=TEXT.encode("utf-8"))
r.raise_for_status()
print(r.text)

9042-5142-1579-1503


In [83]:
session_id = r.text
print(session_id)

9042-5142-1579-1503


In [85]:
url = session_url_template.format(session_id=session_id)
r = requests.get(url)
r.raise_for_status()
print(r.text)

00000|t|Autosomal-dominant striatal degeneration (ADSD) is an autosomal-dominant movement disorder affecting the striatal part of the basal ganglia. ADSD is characterized by bradykinesia, dysarthria, and muscle rigidity. These symptoms resemble idiopathic Parkinson disease, but tremor is not present.  -*- Using genetic linkage analysis, we have mapped the causative genetic defect to a 3.25 megabase candidate region on chromosome 5q13.3-q14.1. A maximum LOD score of 4.1 (Theta = 0) was obtained at marker D5S1962. Here we show that ADSD is caused by a complex frameshift mutation (c.94G>C+c.95delT) in the phosphodiesterase 8B (PDE8B) gene, which results in a loss of enzymatic phosphodiesterase activity. We found that PDE8B is highly expressed in the brain, especially in the putamen, which is affected by ADSD. PDE8B degrades cyclic AMP, a second messenger implied in dopamine signaling. Dopamine is one of the main neurotransmitters involved in movement control and is deficient in Parkinson 

In [86]:
parse_res = r.text
print(parse_res)

00000|t|Autosomal-dominant striatal degeneration (ADSD) is an autosomal-dominant movement disorder affecting the striatal part of the basal ganglia. ADSD is characterized by bradykinesia, dysarthria, and muscle rigidity. These symptoms resemble idiopathic Parkinson disease, but tremor is not present.  -*- Using genetic linkage analysis, we have mapped the causative genetic defect to a 3.25 megabase candidate region on chromosome 5q13.3-q14.1. A maximum LOD score of 4.1 (Theta = 0) was obtained at marker D5S1962. Here we show that ADSD is caused by a complex frameshift mutation (c.94G>C+c.95delT) in the phosphodiesterase 8B (PDE8B) gene, which results in a loss of enzymatic phosphodiesterase activity. We found that PDE8B is highly expressed in the brain, especially in the putamen, which is affected by ADSD. PDE8B degrades cyclic AMP, a second messenger implied in dopamine signaling. Dopamine is one of the main neurotransmitters involved in movement control and is deficient in Parkinson 

## test real data example

In [108]:
# ebi_data = stage1_processing.get_ebi_data()
# 
# ebi_terms = ebi_data["query"].tolist()[:60]
# print(ebi_terms)

from string import punctuation

def _clean(text: str) -> str:
    text = text.encode("ascii", "ignore").decode()
    for _ in punctuation:
        text = text.replace(_, " ")
    return text

efo_data = stage1_processing.get_efo_nodes()
efo_terms = efo_data["efo_label"][600:800].apply(_clean).tolist()
print(efo_terms[:20])

2022-05-24 10:24:08.578 | INFO     | funcs.data_processing.stage1_processing:get_efo_nodes:24 - <class 'pandas.core.frame.DataFrame'>
Int64Index: 25380 entries, 0 to 25389
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   efo_label  25380 non-null  object
 1   efo_id     25380 non-null  object
dtypes: object(2)
memory usage: 594.8+ KB



['eyebrow eyelashes pigmentation anomaly', 'disorder of branched chain amino acid metabolism', 'syndromic keratoconus', 'abnormal eye movements', 'non progressive epilepsy and or ataxia with myoclonus as a major feature', 'x linked recessive optic atrophy', 'hyperlipoproteinemia type 1', 'gm2 gangliosidosis', 'peroxisome biogenesis disorder zellweger syndrome spectrum', 'ichthyosis associated with ocular features', 'cataract associated with a metabolic disease', 'male infertility due to sperm disorder', 'metabolic disease with macular cherry red spot', 'congenital muscular alpha dystroglycanopathy with brain and eye anomalies', 'mucopolysaccharidosis type 1', 'mucopolysaccharidosis', 'phakomatosis with eye involvement', 'craniofacial anomaly with cataract', 'spinocerebellar degenerescence and spastic paraparesis with an oculomotor anomaly', 'hcc1599']


In [105]:
efo_terms_concat = "\n-*-\n".join(efo_terms)
print(efo_terms_concat)

peroxisomal disease with epilepsy
-*-
amino acid or protein metabolism disease with epilepsy
-*-
metal transport or utilization disorder with epilepsy
-*-
secondary interstitial lung disease in childhood and adulthood associated with a metabolic disease
-*-
genetic non syndromic central nervous system malformation
-*-
genetic neurodegenerative disease with dementia
-*-
frontotemporal dementia
-*-
syndromic ichthyosis associated with ocular features
-*-
metabolic disease with corneal opacity
-*-
walker warburg syndrome
-*-
non x linked congenital disorder of glycosylation with intellectual disability as a major feature
-*-
congenital disorder of glycosylation with epilepsy as a major feature
-*-
sphingolipidosis with epilepsy
-*-
inherited renal cancer predisposing syndrome
-*-
syndromic urogenital tract malformation
-*-
trichothiodystrophy
-*-
syndromic hair shaft abnormality
-*-
rare insulin resistance syndrome
-*-
non acquired premature ovarian failure
-*-
optic neuropathy
-*-
famili

In [102]:
url = annotate_url_template.format(concept_type=BIOCONCEPT)
r = requests.post(url, data=efo_terms_concat.encode("utf-8"))
r.raise_for_status()
print(r.text)

7442-7693-5499-3769


In [103]:
session_id = r.text
print(session_id)

7442-7693-5499-3769


In [107]:
url = session_url_template.format(session_id=session_id)
r = requests.get(url)
r.raise_for_status()
print(r.text)

00000|t|peroxisomal disease with epilepsy -*- amino acid or protein metabolism disease with epilepsy -*- metal transport or utilization disorder with epilepsy -*- secondary interstitial lung disease in childhood and adulthood associated with a metabolic disease -*- genetic non syndromic central nervous system malformation -*- genetic neurodegenerative disease with dementia -*- frontotemporal dementia -*- syndromic ichthyosis associated with ocular features -*- metabolic disease with corneal opacity -*- walker warburg syndrome -*- non x linked congenital disorder of glycosylation with intellectual disability as a major feature -*- congenital disorder of glycosylation with epilepsy as a major feature -*- sphingolipidosis with epilepsy -*- inherited renal cancer predisposing syndrome -*- syndromic urogenital tract malformation -*- trichothiodystrophy -*- syndromic hair shaft abnormality -*- rare insulin resistance syndrome -*- non acquired premature ovarian failure -*- optic neuropathy -*

In [75]:
parse_res = r.text.split("\n")

In [76]:
parse_res[2].split("\t")

['00000',
 '0',
 '66',
 'gonarthrosis -*- psoriatic and enteropathic arthropathies -*- pain',
 'Disease',
 'MESH:D015535']